In [1]:
# !python train.py --dataset_name DDR --num_classes 6 --num_train_epochs 1
# !python train.py --dataset_name DDR --num_classes 6 --num_train_epochs 1 --external_embedings True
# !python train.py --dataset_name EyePacs --num_classes 5 --num_train_epochs 15 --run_name EyePacs_15ep
# !python train.py --dataset_name EyePacs --num_classes 5 --num_train_epochs 1 --external_embedings True
# !python train.py --backbone_name resnet50  --dataset_name DDR --run_name DDR05_2__resnet_pre_freeze__SSiT384eyeDdr_short_19 --num_train_epochs 5  --feat_concat --external_embedings

In [1]:
import gc
gc.collect()
import torch
torch.cuda.empty_cache()

In [3]:
# !pip install albumentations
# import torch
# torch.cuda.is_available()

True

In [1]:
!wandb login 1fa58b4e42c64c2531b3abeb43c04f5991be307e
!python train.py --dataset_name DDR --num_train_epochs 2 --run_name Test_1

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
/root/miniconda3/envs/virt_env/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/envs/virt_env/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Train subset resampling ...
0: length: 3068
1: length: 3068
2: length: 3068
3: length: 3068
4: length: 3068
N_added_rows:  6185
N_all_rows:  6245
Ratio of used rows:  0.9903923138510808
/root/miniconda3/envs/virt_env/lib/python3.9/site-packages/transformers/training_a

In [4]:
# !wandb login 1fa58b4e42c64c2531b3abeb43c04f5991be307e

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [1]:
!python test.py --dataset_name DDR --run_name DDR_MLP3_hachiko_extrEmbds_newPrepr   

/root/miniconda3/envs/virt_env/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/envs/virt_env/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
External embedings are used
/workspace/Diabetic-Retinopathy-Classification/model/SSIT/ssit_models.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code duri